In [25]:
import polars as pl
import statsmodels.api as sm
from pathlib import Path
import numpy as np
from sqlalchemy import create_engine
import numba # 引入 Numba JIT 编译器

# =================================================================== #
#                           【1. 数据加载配置】                         #
# =================================================================== #
# --- 数据库连接配置 ---
username = "panjinhe"
password = "20020112p"
host = "localhost"
port = "5432"
database = "pbroe"
table_name = 'pbroech6'
schema_name = 'pbroe'

# --- 日期范围 ---
start_date = '2005-04-01'
end_date = '2025-03-31'

# --- 输出文件配置 ---
OUTPUT_DIR = Path(r'E:\PBROE\ch7pl')
OUTPUT_FILENAME = 'pbroe7.1_residuals_and_quantiles_pure_polars.csv'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True) # 确保输出目录存在


# =================================================================== #
#                       【2. 数据加载 (Polars)】                        #
# =================================================================== #
dfpbroech7 = None
try:
    print("--- 步骤 1: 从PostgreSQL数据库加载数据 ---")
    connection_string = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
    engine = create_engine(connection_string)

    # 在数据库层面直接过滤，效率更高
    sql_query = f"""
    SELECT * FROM {schema_name}.{table_name}
    WHERE "trdmnt" >= '{start_date}' AND "trdmnt" <= '{end_date}'
    """

    # 使用 pl.read_database 执行查询
    dfpbroech7 = pl.read_database(query=sql_query, connection=engine)

    print(f"数据加载成功！共加载 {dfpbroech7.height} 条记录。")
    print("数据概览 (describe):")
    print(dfpbroech7.describe())

except Exception as e:
    print(f"数据加载失败: {e}")
finally:
    if 'engine' in locals() and engine:
        engine.dispose()

# =================================================================== #
#          【3. Numba JIT 加速的滚动分位数计算辅助函数】          #
# =================================================================== #

@numba.jit(nopython=True)
def rolling_rank_get_last_quantile(arr: np.ndarray) -> float:
    """
    一个被 Numba JIT 编译的函数，用于高速计算滚动窗口中最后一个值的百分位排名。
    它接收一个NumPy数组，返回一个浮点数。
    """
    if len(arr) == 0:
        return np.nan

    # 获取窗口中的最后一个值
    last_val = arr[-1]

    # 计算排名（采用 'average' 方法的逻辑）
    less_count = 0
    equal_count = 0
    for x in arr:
        if x < last_val:
            less_count += 1
        elif x == last_val:
            equal_count += 1

    # rank = (小于last_val的数量) + (等于last_val的数量 + 1) / 2
    rank = less_count + (equal_count + 1) / 2.0

    # 返回百分位排名
    return rank / len(arr)


# =================================================================== #
#                  【4. 基于Polars的数据处理主函数】                  #
# =================================================================== #

def process_data_with_polars(df_pl: pl.DataFrame):
    """
    使用纯Polars对输入数据进行完整的处理流程。
    """
    if not isinstance(df_pl, pl.DataFrame) or df_pl.is_empty():
        print("错误: 输入的不是一个有效的Polars DataFrame或DataFrame为空。")
        return

    print("\n--- 步骤 2: 数据清洗与特征工程 (Polars) ---")
    cleaned_df = (
        df_pl
        .filter(pl.col('if_st') != 1)
        .filter(
            (pl.col('PB') > 0) &
            (pl.col('roe_ttm') <= 0.5) & (pl.col('roe_ttm') >= -0.1) &
            (pl.col('roic') <= 0.5) & (pl.col('roic') >= -0.1)
        )
        .drop_nulls(subset=['roe_ttm', 'roic', 'PB', 'indnme1'])
        .with_columns([
            pl.col('PB').log().alias('lnPB'),
            (pl.col('roe_ttm') - pl.col('roic')).alias('leverage_spread')
        ])
    )
    print(f"数据清洗后，剩余 {cleaned_df.height} 条有效记录用于回归分析。")


    print("\n--- 步骤 3: 执行月度行业回归与残差计算 (全向量化) ---")
    # 1. 定义回归变量
    y_col = 'lnPB'
    x1_col = 'roic'
    x2_col = 'leverage_spread'
    group_cols = ['trdmnt', 'indnme1']

    # 2. 在每个分组内，计算OLS回归所需的所有聚合统计量
    stats_df = cleaned_df.group_by(group_cols).agg(
        pl.mean(y_col).alias(f'mean_{y_col}'),
        pl.mean(x1_col).alias(f'mean_{x1_col}'),
        pl.mean(x2_col).alias(f'mean_{x2_col}'),
        pl.var(x1_col).alias(f'var_{x1_col}'),
        pl.var(x2_col).alias(f'var_{x2_col}'),
        pl.cov(x1_col, y_col).alias(f'cov_{x1_col}_{y_col}'),
        pl.cov(x2_col, y_col).alias(f'cov_{x2_col}_{y_col}'),
        pl.cov(x1_col, x2_col).alias(f'cov_{x1_col}_{x2_col}'),
        pl.len().alias('n') # 【代码修正】使用 pl.len() 替代已弃用的 pl.count()
    ).filter(pl.col('n') >= 15)

    # 3. 根据OLS公式，利用聚合统计量计算回归系数
    denominator = (pl.col(f'var_{x1_col}') * pl.col(f'var_{x2_col}')) - (pl.col(f'cov_{x1_col}_{x2_col}') ** 2)
    safe_denominator = pl.when(denominator.abs() < 1e-9).then(1e-9).otherwise(denominator)
    stats_df = stats_df.with_columns(
        b1=((pl.col(f'var_{x2_col}') * pl.col(f'cov_{x1_col}_{y_col}')) - (pl.col(f'cov_{x1_col}_{x2_col}') * pl.col(f'cov_{x2_col}_{y_col}'))) / safe_denominator,
        b2=((pl.col(f'var_{x1_col}') * pl.col(f'cov_{x2_col}_{y_col}')) - (pl.col(f'cov_{x1_col}_{x2_col}') * pl.col(f'cov_{x1_col}_{y_col}'))) / safe_denominator
    ).with_columns(
        b0=pl.col(f'mean_{y_col}') - (pl.col('b1') * pl.col(f'mean_{x1_col}')) - (pl.col('b2') * pl.col(f'mean_{x2_col}'))
    )

    # 4. 将计算出的系数连接回原始数据集
    regression_df = cleaned_df.join(stats_df.select(group_cols + ['b0', 'b1', 'b2']), on=group_cols, how='inner')

    # 5. 计算预测值和残差
    regression_df = regression_df.with_columns(
        predicted_y = pl.col('b0') + (pl.col('b1') * pl.col(x1_col)) + (pl.col('b2') * pl.col(x2_col))
    ).with_columns(
        residual = pl.col(y_col) - pl.col('predicted_y')
    )

    # 6. 计算残差的Z-score标准化值
    regression_results_df = regression_df.with_columns(
        residual_zscore=(
            (pl.col('residual') - pl.col('residual').mean().over(group_cols)) /
            pl.col('residual').std(ddof=0).over(group_cols)
        ).fill_nan(0.0)
    )
    print(f"全向量化回归与残差计算完成，共得到 {regression_results_df.height} 条有效记录。")


    print("\n--- 步骤 4: 计算时序残差分位数 (Numba 加速) ---")
    if regression_results_df.is_empty():
        print("没有有效的残差数据，跳过时序分位数计算。")
        return

    periods = [10]
    df_sorted = regression_results_df.sort('stkcd', 'trdmnt')

    quantile_exprs = []
    for period in periods:
        # =================================================================== #
        #              【代码优化 - 使用 Numba 加速 rolling_map】             #
        # =================================================================== #
        expr = (
            pl.col('residual_zscore')
              .rolling_map(
                  function=lambda s: rolling_rank_get_last_quantile(s.to_numpy()), # 调用高速Numba函数
                  window_size=period,
                  min_samples=1
              )
              .over('stkcd')
              .alias(f'residual_quantile_{period}m')
        )
        quantile_exprs.append(expr)

    df_with_quantiles = df_sorted.with_columns(quantile_exprs)
    print(f"已为 {periods} 周期计算时序残差分位数。")


    print("\n--- 步骤 5: 格式化并保存最终结果 ---")
    final_data = df_with_quantiles.with_columns(
        (pl.col('trdmnt').str.to_date('%Y-%m', strict=False)
         .dt.offset_by('1mo')
         .dt.strftime('%Y-%m-%d'))
        .alias('调入日期')
    )

    output_columns = [
        '调入日期', 'stkcd', 'shortname', 'indnme1',
        'roe_ttm', 'roic', 'PB', 'residual_zscore',
        'residual_quantile_10m'
    ]
    final_output_df = final_data.select([col for col in output_columns if col in final_data.columns])
    output_path = OUTPUT_DIR / OUTPUT_FILENAME
    final_output_df.write_csv(output_path, float_precision=6)
    print(f"\n策略数据已成功生成并保存为 '{output_path}'。")


# =================================================================== #
#                           【5. 执行入口】                           #
# =================================================================== #

if __name__ == '__main__' and dfpbroech7 is not None:
    process_data_with_polars(dfpbroech7)
elif dfpbroech7 is None:
    print("\n数据未能成功加载，处理流程终止。")


--- 步骤 1: 从PostgreSQL数据库加载数据 ---
数据加载成功！共加载 664336 条记录。
数据概览 (describe):
shape: (9, 13)
┌────────────┬────────┬─────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ statistic  ┆ stkcd  ┆ trdmnt  ┆ accper     ┆ … ┆ market_cap ┆ PB         ┆ roe_ttm   ┆ roic      │
│ ---        ┆ ---    ┆ ---     ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│ str        ┆ str    ┆ str     ┆ str        ┆   ┆ f64        ┆ f64        ┆ f64       ┆ f64       │
╞════════════╪════════╪═════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ count      ┆ 664336 ┆ 664336  ┆ 664336     ┆ … ┆ 664336.0   ┆ 662266.0   ┆ 635519.0  ┆ 653819.0  │
│ null_count ┆ 0      ┆ 0       ┆ 0          ┆ … ┆ 0.0        ┆ 2070.0     ┆ 28817.0   ┆ 10517.0   │
│ mean       ┆ null   ┆ null    ┆ 2017-06-06 ┆ … ┆ 1.6867e10  ┆ 4.107753   ┆ 0.056615  ┆ 0.031215  │
│            ┆        ┆         ┆ 12:03:53.3 ┆   ┆            ┆            ┆           ┆           │
│  

In [26]:
# 残差
# pbroe7_backtest_engine_polars.py
# 一个为PB-ROE系列策略设计的、支持分组回测的通用引擎
# 版本：Polars全向量化版 (高性能、高稳定性)

import polars as pl
import numpy as np
from pathlib import Path
import time

# =================================================================== #
#                       【1. 核心回测模块 (Polars)】                    #
# =================================================================== #

def run_grouped_backtest_polars(config):
    """
    (核心函数) 使用 Polars 全向量化方法，对所有分组一次性完成回测。
    """
    print("--- 步骤 1: 加载数据 (Polars) ---")
    try:
        # 加载策略分组数据
        strategy_df = pl.read_csv(config['RESIDUAL_FILE']).with_columns(
            pl.col('调入日期').str.to_date(format='%Y-%m-%d'),
            pl.col('stkcd').cast(pl.Utf8).str.zfill(6)
        )

        # 加载收益率数据
        returns_df = pl.read_csv(config['RETURNS_FILE']).select(
            pl.col('Stkcd').cast(pl.Utf8).str.zfill(6).alias('stkcd'),
            pl.col('Trdmnt').str.to_date(format='%Y-%m').alias('date'),
            pl.col('Mretwd').cast(pl.Float64, strict=False).alias('stock_return')
        )

        # 加载基准数据
        all_benchmarks_df = pl.read_csv(config['BENCHMARK_FILE'])
        benchmark_df = all_benchmarks_df.filter(
            pl.col('Indexcd').cast(pl.Utf8).str.zfill(6) == config['BENCHMARK_CODE']
        ).select(
            pl.col('Month').str.to_date(format='%Y-%m').alias('date'),
            pl.col('Idxrtn').alias('benchmark_return')
        )
        print("所有数据加载成功。")

    except Exception as e:
        print(f"错误: 加载数据时出错: {e}。程序终止。")
        return None, None

    print("\n--- 步骤 2: 构建投资组合并执行回测 (Polars) ---")

    # 1. 创建分组
    num_groups = config['NUM_GROUPS']
    strategy_df = strategy_df.with_columns(
        # 使用 qcut 创建分组，类似 pandas.qcut
        pl.col('residual_zscore').qcut(num_groups, labels=[f"G{i}" for i in range(1, num_groups + 1)])
          .alias('residual_group')
    ).drop_nulls('residual_group').rename({'调入日期': 'date'})

    # 2. 筛选回测周期
    start_date = pl.lit(config['BACKTEST_START_DATE']).str.to_date()
    end_date = pl.lit(config['BACKTEST_END_DATE']).str.to_date()
    strategy_df = strategy_df.filter(pl.col('date').is_between(start_date, end_date))

    # 3. 【核心向量化步骤】将策略数据与收益数据合并
    merged_df = strategy_df.join(returns_df, on=['date', 'stkcd'], how='inner')

    # 4. 【核心向量化步骤】一次性计算所有组在所有月份的平均收益
    monthly_returns = merged_df.group_by(['date', 'residual_group']).agg(
        pl.col('stock_return').mean()
    )

    # 5. 【核心向量化步骤】将结果从长格式转换为宽格式 (pivot)
    #    【代码修正】根据Polars API更新，将 'columns' 参数重命名为 'on'
    portfolio_returns_df = monthly_returns.pivot(
        index='date',
        on='residual_group',
        values='stock_return'
    ).sort('date')

    # 重命名列以匹配后续格式
    new_cols = {'date': 'date'}
    for i in range(1, num_groups + 1):
        if f'G{i}' in portfolio_returns_df.columns:
            new_cols[f'G{i}'] = f'portfolio_return_g{i}'
    portfolio_returns_df = portfolio_returns_df.rename(new_cols)

    # 填充缺失月份，确保时间序列连续
    full_date_range = pl.date_range(
        portfolio_returns_df.get_column('date').min(),
        portfolio_returns_df.get_column('date').max(),
        interval="1mo",
        eager=True
    ).alias("date")

    portfolio_returns_df = pl.DataFrame(full_date_range).join(
        portfolio_returns_df, on='date', how='left'
    ).fill_null(0.0)

    print(f"向量化回测完成，已生成 {len(portfolio_returns_df)} 条月度收益记录。\n")

    return portfolio_returns_df, benchmark_df


# =================================================================== #
#                   【2. 绩效计算与保存 (Polars)】                    #
# =================================================================== #

def calculate_performance_and_save_polars(portfolio_returns_df, benchmark_df, config):
    """为所有分组计算绩效并保存结果。"""
    print("--- 步骤 3: 计算并保存所有分组的绩效 ---")

    output_dir = config['OUTPUT_DIR']
    output_dir.mkdir(parents=True, exist_ok=True)
    returns_output_file = output_dir / f"{config['STRATEGY_NAME']}_returns.csv"
    performance_output_file = output_dir / f"{config['STRATEGY_NAME']}_performance.csv"

    num_groups = config['NUM_GROUPS']
    risk_free_rate = config['RISK_FREE_RATE']

    # 合并基准收益
    final_returns_df = portfolio_returns_df.join(benchmark_df, on='date', how='left').fill_null(0)

    # 计算所有分组的累计收益
    cumulative_exprs = []
    for i in range(1, num_groups + 1):
        return_col = f'portfolio_return_g{i}'
        if return_col in final_returns_df.columns:
            cumulative_exprs.append(
                (1 + pl.col(return_col)).cum_prod().alias(f'cumulative_return_g{i}')
            )
    final_returns_df = final_returns_df.with_columns(cumulative_exprs)

    # 计算各项绩效指标
    all_metrics = []
    total_months = len(final_returns_df)

    # 计算基准的年化收益率，用于后续计算超额收益
    annualized_benchmark_return = ((1 + final_returns_df['benchmark_return']).product() ** (12 / total_months) - 1)

    for group_id in range(1, num_groups + 1):
        return_col = f'portfolio_return_g{group_id}'
        cum_return_col = f'cumulative_return_g{group_id}'
        if return_col not in final_returns_df.columns: continue

        # 使用 Polars 表达式一次性计算所有指标
        group_perf = final_returns_df.select(
            pl.col(return_col).alias('return'),
            pl.col(cum_return_col).alias('cum_return'),
            (pl.col(return_col) - pl.col('benchmark_return')).alias('excess_return')
        ).select(
            annualized_return = ((pl.col('cum_return').last()) ** (12 / total_months) - 1),
            annualized_volatility = (pl.col('return').std() * np.sqrt(12)),
            max_drawdown = (((pl.col('cum_return') / pl.col('cum_return').cum_max()) - 1).min()),
            tracking_error = (pl.col('excess_return').std() * np.sqrt(12))
        ).row(0, named=True)

        annualized_return = group_perf['annualized_return']
        annualized_volatility = group_perf['annualized_volatility']
        sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility if annualized_volatility != 0 else 0
        annualized_excess_return = annualized_return - annualized_benchmark_return
        tracking_error = group_perf['tracking_error']
        information_ratio = annualized_excess_return / tracking_error if tracking_error != 0 else 0

        metrics = {
            'group': f"Group {group_id}",
            '年化收益率': annualized_return,
            '年化波动率': annualized_volatility,
            '夏普比率': sharpe_ratio,
            '最大回撤': group_perf['max_drawdown'],
            '累计收益率': final_returns_df[cum_return_col].last() - 1,
            '年化超额收益率': annualized_excess_return,
            '信息比率': information_ratio,
            '跟踪误差': tracking_error
        }
        all_metrics.append(metrics)

    performance_df = pl.DataFrame(all_metrics)

    # 【代码修正】为基准计算完整的绩效指标，并创建一个形状匹配的DataFrame
    benchmark_perf = final_returns_df.select(
        pl.col('benchmark_return').alias('return')
    ).with_columns(
        (1 + pl.col('return')).cum_prod().alias('cum_return')
    ).select(
        annualized_return = (((pl.col('cum_return').last()) ** (12 / total_months)) - 1),
        annualized_volatility = (pl.col('return').std() * np.sqrt(12)),
        max_drawdown = (((pl.col('cum_return') / pl.col('cum_return').cum_max()) - 1).min()),
        total_cumulative_return = pl.col('cum_return').last()
    ).row(0, named=True)

    benchmark_row_data = {
        'group': '基准 (沪深300)',
        '年化收益率': benchmark_perf['annualized_return'],
        '年化波动率': benchmark_perf['annualized_volatility'],
        '夏普比率': (benchmark_perf['annualized_return'] - risk_free_rate) / benchmark_perf['annualized_volatility'] if benchmark_perf['annualized_volatility'] != 0 else 0,
        '最大回撤': benchmark_perf['max_drawdown'],
        '累计收益率': benchmark_perf['total_cumulative_return'] - 1,
        '年化超额收益率': None,
        '信息比率': None,
        '跟踪误差': None
    }
    benchmark_row = pl.DataFrame([benchmark_row_data])

    # 使用 concat 进行安全的合并
    performance_df = pl.concat([performance_df, benchmark_row], how='vertical')

    final_returns_df.write_csv(returns_output_file, float_precision=6)
    print(f"\n所有分组的月度收益率详情已保存至: {returns_output_file}")
    performance_df.write_csv(performance_output_file, float_precision=6)
    print(f"所有分组的绩效指标已保存至: {performance_output_file}")
    print(f"\n--- {config['STRATEGY_NAME']} 各分组绩效简报 ---")
    print(performance_df)


# =================================================================== #
#                          【3. 主函数执行】                          #
# =================================================================== #

def main(config):
    """主执行函数"""
    start_time = time.time()

    # 步骤 1 & 2: 执行全向量化回测
    portfolio_returns_df, benchmark_df = run_grouped_backtest_polars(config)

    if portfolio_returns_df is None:
        print("回测失败，程序终止。")
        return

    # 步骤 3: 计算并保存绩效
    calculate_performance_and_save_polars(portfolio_returns_df, benchmark_df, config)

    end_time = time.time()
    print(f"\n--- 所有任务完成！总耗时: {end_time - start_time:.2f} 秒 ---")

# =================================================================== #
#                       【4. 脚本执行入口】                              #
# =================================================================== #

if __name__ == "__main__":

    # --- 定义 pbroe7.1 分组回测的配置 ---
    CONFIG_PBROE7_GROUPED = {
        # --- 策略与输出配置 ---
        "STRATEGY_NAME": "pbroe7.1_grouped_backtest_polars",
        "OUTPUT_DIR": Path("E:/PBROE/ch7pl/backtest_results_polars"),

        # --- 输入文件路径 ---
        "RESIDUAL_FILE": Path("E:/PBROE/ch7pl/pbroe7.1_residuals_and_quantiles_pure_polars.csv"),
        "RETURNS_FILE": Path("E:/PBROE/data/TRDNEW_Mnth.csv"),
        "BENCHMARK_FILE": Path("E:/PBROE/data/benchmark_indices.csv"),

        # --- 策略核心参数 ---
        "NUM_GROUPS": 10,

        # --- 通用回测参数 ---
        "BACKTEST_START_DATE": '2010-05-01',
        "BACKTEST_END_DATE": '2025-04-30',
        "BENCHMARK_CODE": '000300',
        "RISK_FREE_RATE": 0.03
    }

    # --- 执行回测 ---
    # 调用主函数，并传入配置字典
    main(CONFIG_PBROE7_GROUPED)


--- 步骤 1: 加载数据 (Polars) ---
所有数据加载成功。

--- 步骤 2: 构建投资组合并执行回测 (Polars) ---
向量化回测完成，已生成 180 条月度收益记录。

--- 步骤 3: 计算并保存所有分组的绩效 ---

所有分组的月度收益率详情已保存至: E:\PBROE\ch7pl\backtest_results_polars\pbroe7.1_grouped_backtest_polars_returns.csv
所有分组的绩效指标已保存至: E:\PBROE\ch7pl\backtest_results_polars\pbroe7.1_grouped_backtest_polars_performance.csv

--- pbroe7.1_grouped_backtest_polars 各分组绩效简报 ---
shape: (11, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ group     ┆ 年化收益  ┆ 年化波动  ┆ 夏普比率  ┆ … ┆ 累计收益  ┆ 年化超额  ┆ 信息比率  ┆ 跟踪误差 │
│ ---       ┆ 率        ┆ 率        ┆ ---       ┆   ┆ 率        ┆ 收益率    ┆ ---       ┆ ---      │
│ str       ┆ ---       ┆ ---       ┆ f64       ┆   ┆ ---       ┆ ---       ┆ f64       ┆ f64      │
│           ┆ f64       ┆ f64       ┆           ┆   ┆ f64       ┆ f64       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Group 1   ┆ 0.150373  ┆ 0.25

In [31]:
# 分位数残差
# pbroe7_backtest_engine_polars.py
# 一个为PB-ROE系列策略设计的、支持分组回测的通用引擎
# 版本：Polars全向量化版 (高性能、高稳定性)

import polars as pl
import numpy as np
from pathlib import Path
import time

# =================================================================== #
#                       【1. 核心回测模块 (Polars)】                    #
# =================================================================== #

def run_grouped_backtest_polars(config):
    """
    (核心函数) 使用 Polars 全向量化方法，对所有分组一次性完成回测。
    """
    print("--- 步骤 1: 加载数据 (Polars) ---")
    try:
        # 加载策略分组数据
        strategy_df = pl.read_csv(config['RESIDUAL_FILE']).with_columns(
            pl.col('调入日期').str.to_date(format='%Y-%m-%d'),
            pl.col('stkcd').cast(pl.Utf8).str.zfill(6)
        )

        # 加载收益率数据
        returns_df = pl.read_csv(config['RETURNS_FILE']).select(
            pl.col('Stkcd').cast(pl.Utf8).str.zfill(6).alias('stkcd'),
            pl.col('Trdmnt').str.to_date(format='%Y-%m').alias('date'),
            pl.col('Mretwd').cast(pl.Float64, strict=False).alias('stock_return')
        )

        # 加载基准数据
        all_benchmarks_df = pl.read_csv(config['BENCHMARK_FILE'])
        benchmark_df = all_benchmarks_df.filter(
            pl.col('Indexcd').cast(pl.Utf8).str.zfill(6) == config['BENCHMARK_CODE']
        ).select(
            pl.col('Month').str.to_date(format='%Y-%m').alias('date'),
            pl.col('Idxrtn').alias('benchmark_return')
        )
        print("所有数据加载成功。")

    except Exception as e:
        print(f"错误: 加载数据时出错: {e}。程序终止。")
        return None, None

    print("\n--- 步骤 2: 构建投资组合并执行回测 (Polars) ---")

    # 1. 创建分组
    num_groups = config['NUM_GROUPS']
    grouping_col = config['GROUPING_COLUMN'] # 从配置中读取分组列

    # =================================================================== #
    #         【代码修正 - 使用 when/then 手动实现 cut】         #
    # =================================================================== #
    # 由于 cut 函数在不同 Polars 版本中 API 不稳定，我们使用更稳健的
    # when/then 表达式链来手动实现精确的区间分组。

    # 动态构建 when/then 表达式链
    # (0, 0.1] -> G1, (0.1, 0.2] -> G2, ...
    # when/then 链会按顺序执行，第一个满足条件的分支会被采纳。
    grouping_expr = pl.when(pl.col(grouping_col) <= (1 / num_groups)).then(pl.lit("G1"))
    for i in range(2, num_groups + 1):
        upper_bound = i / num_groups
        grouping_expr = grouping_expr.when(pl.col(grouping_col) <= upper_bound).then(pl.lit(f"G{i}"))

    strategy_df = strategy_df.with_columns(
        grouping_expr.otherwise(pl.lit(None)).alias('residual_group')
    ).drop_nulls('residual_group').rename({'调入日期': 'date'})


    # 2. 筛选回测周期
    start_date = pl.lit(config['BACKTEST_START_DATE']).str.to_date()
    end_date = pl.lit(config['BACKTEST_END_DATE']).str.to_date()
    strategy_df = strategy_df.filter(pl.col('date').is_between(start_date, end_date))

    # 3. 【核心向量化步骤】将策略数据与收益数据合并
    merged_df = strategy_df.join(returns_df, on=['date', 'stkcd'], how='inner')

    # 4. 【核心向量化步骤】一次性计算所有组在所有月份的平均收益
    monthly_returns = merged_df.group_by(['date', 'residual_group']).agg(
        pl.col('stock_return').mean()
    )

    # 5. 【核心向量化步骤】将结果从长格式转换为宽格式 (pivot)
    portfolio_returns_df = monthly_returns.pivot(
        index='date',
        on='residual_group',
        values='stock_return'
    ).sort('date')

    # 重命名列以匹配后续格式
    new_cols = {'date': 'date'}
    for i in range(1, num_groups + 1):
        if f"G{i}" in portfolio_returns_df.columns:
            new_cols[f"G{i}"] = f'portfolio_return_g{i}'
    portfolio_returns_df = portfolio_returns_df.rename(new_cols)

    # 填充缺失月份，确保时间序列连续
    if portfolio_returns_df.is_empty():
        print("警告：在指定的回测周期内没有生成任何投资组合收益。")
        return None, None

    full_date_range = pl.date_range(
        portfolio_returns_df.get_column('date').min(),
        portfolio_returns_df.get_column('date').max(),
        interval="1mo",
        eager=True
    ).alias("date")

    portfolio_returns_df = pl.DataFrame(full_date_range).join(
        portfolio_returns_df, on='date', how='left'
    ).fill_null(0.0)

    print(f"向量化回测完成，已生成 {len(portfolio_returns_df)} 条月度收益记录。\n")

    return portfolio_returns_df, benchmark_df


# =================================================================== #
#                   【2. 绩效计算与保存 (Polars)】                    #
# =================================================================== #

def calculate_performance_and_save_polars(portfolio_returns_df, benchmark_df, config):
    """为所有分组计算绩效并保存结果。"""
    print("--- 步骤 3: 计算并保存所有分组的绩效 ---")

    output_dir = config['OUTPUT_DIR']
    output_dir.mkdir(parents=True, exist_ok=True)
    returns_output_file = output_dir / f"{config['STRATEGY_NAME']}_returns.csv"
    performance_output_file = output_dir / f"{config['STRATEGY_NAME']}_performance.csv"

    num_groups = config['NUM_GROUPS']
    risk_free_rate = config['RISK_FREE_RATE']

    # 合并基准收益
    final_returns_df = portfolio_returns_df.join(benchmark_df, on='date', how='left').fill_null(0)

    # 计算所有分组的累计收益
    cumulative_exprs = []
    for i in range(1, num_groups + 1):
        return_col = f'portfolio_return_g{i}'
        if return_col in final_returns_df.columns:
            cumulative_exprs.append(
                (1 + pl.col(return_col)).cum_prod().alias(f'cumulative_return_g{i}')
            )
    final_returns_df = final_returns_df.with_columns(cumulative_exprs)

    # 计算各项绩效指标
    all_metrics = []
    total_months = len(final_returns_df)

    if total_months == 0:
        print("错误：无法计算绩效，因为没有有效的月度收益数据。")
        return

    # 计算基准的年化收益率，用于后续计算超额收益
    annualized_benchmark_return = ((1 + final_returns_df['benchmark_return']).product() ** (12 / total_months) - 1)

    for group_id in range(1, num_groups + 1):
        return_col = f'portfolio_return_g{group_id}'
        cum_return_col = f'cumulative_return_g{group_id}'
        if return_col not in final_returns_df.columns: continue

        # 使用 Polars 表达式一次性计算所有指标
        group_perf = final_returns_df.select(
            pl.col(return_col).alias('return'),
            pl.col(cum_return_col).alias('cum_return'),
            (pl.col(return_col) - pl.col('benchmark_return')).alias('excess_return')
        ).select(
            annualized_return = ((pl.col('cum_return').last()) ** (12 / total_months) - 1),
            annualized_volatility = (pl.col('return').std() * np.sqrt(12)),
            max_drawdown = (((pl.col('cum_return') / pl.col('cum_return').cum_max()) - 1).min()),
            tracking_error = (pl.col('excess_return').std() * np.sqrt(12))
        ).row(0, named=True)

        annualized_return = group_perf['annualized_return']
        annualized_volatility = group_perf['annualized_volatility']
        sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility if annualized_volatility != 0 else 0
        annualized_excess_return = annualized_return - annualized_benchmark_return
        tracking_error = group_perf['tracking_error']
        information_ratio = annualized_excess_return / tracking_error if tracking_error != 0 else 0

        metrics = {
            'group': f"Group {group_id}",
            '年化收益率': annualized_return,
            '年化波动率': annualized_volatility,
            '夏普比率': sharpe_ratio,
            '最大回撤': group_perf['max_drawdown'],
            '累计收益率': final_returns_df[cum_return_col].last() - 1,
            '年化超额收益率': annualized_excess_return,
            '信息比率': information_ratio,
            '跟踪误差': tracking_error
        }
        all_metrics.append(metrics)

    performance_df = pl.DataFrame(all_metrics)

    # 为基准计算完整的绩效指标
    benchmark_perf = final_returns_df.select(
        pl.col('benchmark_return').alias('return')
    ).with_columns(
        (1 + pl.col('return')).cum_prod().alias('cum_return')
    ).select(
        annualized_return = (((pl.col('cum_return').last()) ** (12 / total_months)) - 1),
        annualized_volatility = (pl.col('return').std() * np.sqrt(12)),
        max_drawdown = (((pl.col('cum_return') / pl.col('cum_return').cum_max()) - 1).min()),
        total_cumulative_return = pl.col('cum_return').last()
    ).row(0, named=True)

    benchmark_row_data = {
        'group': '基准 (沪深300)',
        '年化收益率': benchmark_perf['annualized_return'],
        '年化波动率': benchmark_perf['annualized_volatility'],
        '夏普比率': (benchmark_perf['annualized_return'] - risk_free_rate) / benchmark_perf['annualized_volatility'] if benchmark_perf['annualized_volatility'] != 0 else 0,
        '最大回撤': benchmark_perf['max_drawdown'],
        '累计收益率': benchmark_perf['total_cumulative_return'] - 1,
        '年化超额收益率': None,
        '信息比率': None,
        '跟踪误差': None
    }
    benchmark_row = pl.DataFrame([benchmark_row_data])

    # 使用 concat 进行安全的合并
    performance_df = pl.concat([performance_df, benchmark_row], how='vertical')

    final_returns_df.write_csv(returns_output_file, float_precision=6)
    print(f"\n所有分组的月度收益率详情已保存至: {returns_output_file}")
    performance_df.write_csv(performance_output_file, float_precision=6)
    print(f"所有分组的绩效指标已保存至: {performance_output_file}")
    print(f"\n--- {config['STRATEGY_NAME']} 各分组绩效简报 ---")
    print(performance_df)


# =================================================================== #
#                          【3. 主函数执行】                          #
# =================================================================== #

def main(config):
    """主执行函数"""
    start_time = time.time()

    # 步骤 1 & 2: 执行全向量化回测
    portfolio_returns_df, benchmark_df = run_grouped_backtest_polars(config)

    if portfolio_returns_df is None:
        print("回测失败，程序终止。")
        return

    # 步骤 3: 计算并保存绩效
    calculate_performance_and_save_polars(portfolio_returns_df, benchmark_df, config)

    end_time = time.time()
    print(f"\n--- 所有任务完成！总耗时: {end_time - start_time:.2f} 秒 ---")

# =================================================================== #
#                       【4. 脚本执行入口】                              #
# =================================================================== #

if __name__ == "__main__":

    # --- 定义 pbroe7.2 时序分位数回测的配置 ---
    CONFIG_PBROE7_QUANTILE = {
        # --- 策略与输出配置 ---
        "STRATEGY_NAME": "pbroe7.2_quantile_backtest_polars",
        "OUTPUT_DIR": Path("E:/PBROE/ch7pl/backtest_results_polars"),

        # --- 输入文件路径 ---
        "RESIDUAL_FILE": Path("E:/PBROE/ch7pl/pbroe7.1_residuals_and_quantiles_pure_polars.csv"),
        "RETURNS_FILE": Path("E:/PBROE/data/TRDNEW_Mnth.csv"),
        "BENCHMARK_FILE": Path("E:/PBROE/data/benchmark_indices.csv"),

        # --- 策略核心参数 ---
        "GROUPING_COLUMN": "residual_quantile_10m", # 明确指定分组列
        "NUM_GROUPS": 10,

        # --- 通用回测参数 ---
        "BACKTEST_START_DATE": '2010-05-01',
        "BACKTEST_END_DATE": '2025-04-30',
        "BENCHMARK_CODE": '000300',
        "RISK_FREE_RATE": 0.03
    }

    # --- 执行回测 ---
    # 调用主函数，并传入配置字典
    main(CONFIG_PBROE7_QUANTILE)


--- 步骤 1: 加载数据 (Polars) ---
所有数据加载成功。

--- 步骤 2: 构建投资组合并执行回测 (Polars) ---
向量化回测完成，已生成 180 条月度收益记录。

--- 步骤 3: 计算并保存所有分组的绩效 ---

所有分组的月度收益率详情已保存至: E:\PBROE\ch7pl\backtest_results_polars\pbroe7.2_quantile_backtest_polars_returns.csv
所有分组的绩效指标已保存至: E:\PBROE\ch7pl\backtest_results_polars\pbroe7.2_quantile_backtest_polars_performance.csv

--- pbroe7.2_quantile_backtest_polars 各分组绩效简报 ---
shape: (11, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ group     ┆ 年化收益  ┆ 年化波动  ┆ 夏普比率  ┆ … ┆ 累计收益  ┆ 年化超额  ┆ 信息比率  ┆ 跟踪误差 │
│ ---       ┆ 率        ┆ 率        ┆ ---       ┆   ┆ 率        ┆ 收益率    ┆ ---       ┆ ---      │
│ str       ┆ ---       ┆ ---       ┆ f64       ┆   ┆ ---       ┆ ---       ┆ f64       ┆ f64      │
│           ┆ f64       ┆ f64       ┆           ┆   ┆ f64       ┆ f64       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Group 1   ┆ 0.182689  ┆ 0

In [40]:
# 交集
# pbroe7_intersection_backtest_polars.py
# 一个为PB-ROE系列策略设计的、专门用于回测双因子交集的引擎
# 版本：Polars全向量化版 (高性能、高稳定性)

import polars as pl
import numpy as np
from pathlib import Path
import time

# =================================================================== #
#                       【1. 核心回测模块 (Polars)】                    #
# =================================================================== #

def run_intersection_backtest_polars(config):
    """
    (核心函数) 使用 Polars 全向量化方法，回测双因子G1组的交集。
    """
    print("--- 步骤 1: 加载数据 (Polars) ---")
    try:
        # 加载策略分组数据
        strategy_df = pl.read_csv(config['RESIDUAL_FILE']).with_columns(
            pl.col('调入日期').str.to_date(format='%Y-%m-%d'),
            pl.col('stkcd').cast(pl.Utf8).str.zfill(6)
        )

        # 加载收益率数据
        returns_df = pl.read_csv(config['RETURNS_FILE']).select(
            pl.col('Stkcd').cast(pl.Utf8).str.zfill(6).alias('stkcd'),
            pl.col('Trdmnt').str.to_date(format='%Y-%m').alias('date'),
            pl.col('Mretwd').cast(pl.Float64, strict=False).alias('stock_return')
        )

        # 加载基准数据
        all_benchmarks_df = pl.read_csv(config['BENCHMARK_FILE'])
        benchmark_df = all_benchmarks_df.filter(
            pl.col('Indexcd').cast(pl.Utf8).str.zfill(6) == config['BENCHMARK_CODE']
        ).select(
            pl.col('Month').str.to_date(format='%Y-%m').alias('date'),
            pl.col('Idxrtn').alias('benchmark_return')
        )
        print("所有数据加载成功。")

    except Exception as e:
        print(f"错误: 加载数据时出错: {e}。程序终止。")
        return None, None

    print("\n--- 步骤 2: 构建交集投资组合 ---")

    # 1. 构建 residual_zscore 的 G1 组合
    #    使用 qcut 进行横截面分组，选出残差最低的10%
    df_zscore_g1 = strategy_df.with_columns(
        group_z = pl.col('residual_zscore').qcut(10, labels=[f"G{i}" for i in range(1, 11)])
    ).filter(pl.col('group_z') == 'G1').rename({'调入日期': 'date'})
    print(f"已构建 Z-Score G1 组合，共 {len(df_zscore_g1)} 条记录。")

    # 2. 构建 residual_quantile_10m 的 G1 组合
    #    使用 when/then 进行时序分位数分组，选出分位数在 (0, 0.1] 区间的股票
    df_quantile_g1 = strategy_df.with_columns(
        group_q = pl.when(pl.col('residual_quantile_10m') <= 0.1).then(pl.lit("G1"))
    ).filter(pl.col('group_q') == 'G1').rename({'调入日期': 'date'})
    print(f"已构建 Quantile G1 组合，共 {len(df_quantile_g1)} 条记录。")

    # 3. 找出两个 G1 组合在每个调仓日的交集
    intersection_portfolio = df_zscore_g1.join(
        df_quantile_g1.select(['date', 'stkcd']), # 只需关键列即可
        on=['date', 'stkcd'],
        how='inner'
    )
    print(f"已构建交集组合，共 {len(intersection_portfolio)} 条记录。")

    # 4. 筛选回测周期
    start_date = pl.lit(config['BACKTEST_START_DATE']).str.to_date()
    end_date = pl.lit(config['BACKTEST_END_DATE']).str.to_date()
    intersection_portfolio = intersection_portfolio.filter(pl.col('date').is_between(start_date, end_date))

    # 5. 【核心向量化步骤】将交集组合与收益数据合并
    merged_df = intersection_portfolio.join(returns_df, on=['date', 'stkcd'], how='inner')

    # 6. 【核心向量化步骤】计算交集组合的等权月度收益
    portfolio_returns_df = merged_df.group_by('date').agg(
        pl.col('stock_return').mean().alias('portfolio_return')
    ).sort('date')

    # 填充缺失月份，确保时间序列连续
    if portfolio_returns_df.is_empty():
        print("警告：在指定的回测周期内没有生成任何投资组合收益。")
        return None, None

    full_date_range = pl.date_range(
        portfolio_returns_df.get_column('date').min(),
        portfolio_returns_df.get_column('date').max(),
        interval="1mo",
        eager=True
    ).alias("date")

    portfolio_returns_df = pl.DataFrame(full_date_range).join(
        portfolio_returns_df, on='date', how='left'
    ).fill_null(0.0)

    print(f"向量化回测完成，已生成 {len(portfolio_returns_df)} 条月度收益记录。\n")

    return portfolio_returns_df, benchmark_df


# =================================================================== #
#                   【2. 绩效计算与保存 (Polars)】                    #
# =================================================================== #

def calculate_performance_and_save_polars(portfolio_returns_df, benchmark_df, config):
    """为交集组合计算绩效并保存结果。"""
    print("--- 步骤 3: 计算并保存交集组合的绩效 ---")

    output_dir = config['OUTPUT_DIR']
    output_dir.mkdir(parents=True, exist_ok=True)
    returns_output_file = output_dir / f"{config['STRATEGY_NAME']}_returns.csv"
    performance_output_file = output_dir / f"{config['STRATEGY_NAME']}_performance.csv"

    risk_free_rate = config['RISK_FREE_RATE']

    # 合并基准收益
    final_returns_df = portfolio_returns_df.join(benchmark_df, on='date', how='left').fill_null(0)

    # 计算组合和基准的累计收益
    final_returns_df = final_returns_df.with_columns(
        (1 + pl.col('portfolio_return')).cum_prod().alias('cumulative_portfolio_return'),
        (1 + pl.col('benchmark_return')).cum_prod().alias('cumulative_benchmark_return')
    )

    # 计算各项绩效指标
    total_months = len(final_returns_df)
    if total_months == 0:
        print("错误：无法计算绩效，因为没有有效的月度收益数据。")
        return

    # 计算组合指标
    portfolio_perf = final_returns_df.select(
        pl.col('portfolio_return').alias('return'),
        pl.col('cumulative_portfolio_return').alias('cum_return'),
        (pl.col('portfolio_return') - pl.col('benchmark_return')).alias('excess_return')
    ).select(
        annualized_return = ((pl.col('cum_return').last()) ** (12 / total_months) - 1),
        annualized_volatility = (pl.col('return').std() * np.sqrt(12)),
        max_drawdown = (((pl.col('cum_return') / pl.col('cum_return').cum_max()) - 1).min()),
        tracking_error = (pl.col('excess_return').std() * np.sqrt(12))
    ).row(0, named=True)

    # 计算基准指标
    benchmark_perf = final_returns_df.select(
        pl.col('benchmark_return').alias('return'),
        pl.col('cumulative_benchmark_return').alias('cum_return')
    ).select(
        annualized_return = ((pl.col('cum_return').last()) ** (12 / total_months) - 1),
        annualized_volatility = (pl.col('return').std() * np.sqrt(12)),
        max_drawdown = (((pl.col('cum_return') / pl.col('cum_return').cum_max()) - 1).min())
    ).row(0, named=True)

    annualized_excess_return = portfolio_perf['annualized_return'] - benchmark_perf['annualized_return']
    tracking_error = portfolio_perf['tracking_error']
    information_ratio = annualized_excess_return / tracking_error if tracking_error != 0 else 0

    # 整理并合并结果
    all_metrics = [
        {
            'group': '交集策略',
            '年化收益率': portfolio_perf['annualized_return'],
            '年化波动率': portfolio_perf['annualized_volatility'],
            '夏普比率': (portfolio_perf['annualized_return'] - risk_free_rate) / portfolio_perf['annualized_volatility'] if portfolio_perf['annualized_volatility'] != 0 else 0,
            '最大回撤': portfolio_perf['max_drawdown'],
            '累计收益率': final_returns_df['cumulative_portfolio_return'].last() - 1,
            '年化超额收益率': annualized_excess_return,
            '信息比率': information_ratio,
            '跟踪误差': tracking_error
        },
        {
            'group': '基准 (沪深300)',
            '年化收益率': benchmark_perf['annualized_return'],
            '年化波动率': benchmark_perf['annualized_volatility'],
            '夏普比率': (benchmark_perf['annualized_return'] - risk_free_rate) / benchmark_perf['annualized_volatility'] if benchmark_perf['annualized_volatility'] != 0 else 0,
            '最大回撤': benchmark_perf['max_drawdown'],
            '累计收益率': final_returns_df['cumulative_benchmark_return'].last() - 1,
            '年化超额收益率': None, '信息比率': None, '跟踪误差': None
        }
    ]
    performance_df = pl.DataFrame(all_metrics)

    final_returns_df.write_csv(returns_output_file, float_precision=6)
    print(f"\n交集组合的月度收益率详情已保存至: {returns_output_file}")
    performance_df.write_csv(performance_output_file, float_precision=6)
    print(f"交集组合的绩效指标已保存至: {performance_output_file}")
    print(f"\n--- {config['STRATEGY_NAME']} 绩效简报 ---")
    print(performance_df)


# =================================================================== #
#                          【3. 主函数执行】                          #
# =================================================================== #

def main(config):
    """主执行函数"""
    start_time = time.time()
    portfolio_returns_df, benchmark_df = run_intersection_backtest_polars(config)
    if portfolio_returns_df is not None:
        calculate_performance_and_save_polars(portfolio_returns_df, benchmark_df, config)
    end_time = time.time()
    print(f"\n--- 所有任务完成！总耗时: {end_time - start_time:.2f} 秒 ---")

# =================================================================== #
#                       【4. 脚本执行入口】                              #
# =================================================================== #

if __name__ == "__main__":

    # --- 定义 pbroe7.3 交集策略回测的配置 ---
    CONFIG_INTERSECTION = {
        # --- 策略与输出配置 ---
        "STRATEGY_NAME": "pbroe7.3_intersection_backtest_polars",
        "OUTPUT_DIR": Path("E:/PBROE/ch7pl/backtest_results_polars"),

        # --- 输入文件路径 ---
        "RESIDUAL_FILE": Path("E:/PBROE/ch7pl/pbroe7.1_residuals_and_quantiles_pure_polars.csv"),
        "RETURNS_FILE": Path("E:/PBROE/data/TRDNEW_Mnth.csv"),
        "BENCHMARK_FILE": Path("E:/PBROE/data/benchmark_indices.csv"),

        # --- 通用回测参数 ---
        "BACKTEST_START_DATE": '2010-05-01',
        "BACKTEST_END_DATE": '2025-04-30',
        "BENCHMARK_CODE": '000300',
        "RISK_FREE_RATE": 0.03
    }

    # --- 执行回测 ---
    main(CONFIG_INTERSECTION)


--- 步骤 1: 加载数据 (Polars) ---
所有数据加载成功。

--- 步骤 2: 构建交集投资组合 ---
已构建 Z-Score G1 组合，共 52740 条记录。
已构建 Quantile G1 组合，共 83307 条记录。
已构建交集组合，共 15121 条记录。
向量化回测完成，已生成 180 条月度收益记录。

--- 步骤 3: 计算并保存交集组合的绩效 ---

交集组合的月度收益率详情已保存至: E:\PBROE\ch7pl\backtest_results_polars\pbroe7.3_intersection_backtest_polars_returns.csv
交集组合的绩效指标已保存至: E:\PBROE\ch7pl\backtest_results_polars\pbroe7.3_intersection_backtest_polars_performance.csv

--- pbroe7.3_intersection_backtest_polars 绩效简报 ---
shape: (2, 9)
┌────────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬──────────┐
│ group      ┆ 年化收益  ┆ 年化波动  ┆ 夏普比率  ┆ … ┆ 累计收益  ┆ 年化超额  ┆ 信息比率 ┆ 跟踪误差 │
│ ---        ┆ 率        ┆ 率        ┆ ---       ┆   ┆ 率        ┆ 收益率    ┆ ---      ┆ ---      │
│ str        ┆ ---       ┆ ---       ┆ f64       ┆   ┆ ---       ┆ ---       ┆ f64      ┆ f64      │
│            ┆ f64       ┆ f64       ┆           ┆   ┆ f64       ┆ f64       ┆          ┆          │
╞════════════╪═══════════╪═══════════╪══════════

In [41]:
# pbroe7_intersection_backtest_polars.py
# 一个为PB-ROE系列策略设计的、专门用于回测双因子交集的引擎
# 版本：Polars全向量化版 (高性能、高稳定性)

import polars as pl
import numpy as np
from pathlib import Path
import time

# =================================================================== #
#                       【1. 核心回测模块 (Polars)】                    #
# =================================================================== #

def run_intersection_backtest_polars(config):
    """
    (核心函数) 使用 Polars 全向量化方法，回测双因子交集。
    """
    print("--- 步骤 1: 加载数据 (Polars) ---")
    try:
        # 加载策略分组数据
        strategy_df = pl.read_csv(config['RESIDUAL_FILE']).with_columns(
            pl.col('调入日期').str.to_date(format='%Y-%m-%d'),
            pl.col('stkcd').cast(pl.Utf8).str.zfill(6)
        )

        # 加载收益率数据
        returns_df = pl.read_csv(config['RETURNS_FILE']).select(
            pl.col('Stkcd').cast(pl.Utf8).str.zfill(6).alias('stkcd'),
            pl.col('Trdmnt').str.to_date(format='%Y-%m').alias('date'),
            pl.col('Mretwd').cast(pl.Float64, strict=False).alias('stock_return')
        )

        # 加载基准数据
        all_benchmarks_df = pl.read_csv(config['BENCHMARK_FILE'])
        benchmark_df = all_benchmarks_df.filter(
            pl.col('Indexcd').cast(pl.Utf8).str.zfill(6) == config['BENCHMARK_CODE']
        ).select(
            pl.col('Month').str.to_date(format='%Y-%m').alias('date'),
            pl.col('Idxrtn').alias('benchmark_return')
        )
        print("所有数据加载成功。")

    except Exception as e:
        print(f"错误: 加载数据时出错: {e}。程序终止。")
        return None, None

    print("\n--- 步骤 2: 构建交集投资组合 ---")

    # =================================================================== #
    #                【代码修正 - 调整筛选标准】                #
    # =================================================================== #
    # 1. 构建 residual_zscore 的前 20% 组合
    #    使用 qcut(5, ...) 进行五等分分组，选出残差最低的20%
    df_zscore_g1 = strategy_df.with_columns(
        group_z = pl.col('residual_zscore').qcut(5, labels=[f"G{i}" for i in range(1, 6)])
    ).filter(pl.col('group_z') == 'G1').rename({'调入日期': 'date'})
    print(f"已构建 Z-Score 最低 20% 组合，共 {len(df_zscore_g1)} 条记录。")

    # 2. 构建 residual_quantile_10m 的前 10% 组合
    #    选出时序分位数在 (0, 0.1] 区间的股票
    df_quantile_g1 = strategy_df.with_columns(
        group_q = pl.when(pl.col('residual_quantile_10m') <= 0.1).then(pl.lit("G1"))
    ).filter(pl.col('group_q') == 'G1').rename({'调入日期': 'date'})
    print(f"已构建 Quantile 最低 10% 组合，共 {len(df_quantile_g1)} 条记录。")

    # 3. 找出两个组合在每个调仓日的交集
    intersection_portfolio = df_zscore_g1.join(
        df_quantile_g1.select(['date', 'stkcd']), # 只需关键列即可
        on=['date', 'stkcd'],
        how='inner'
    )
    print(f"已构建交集组合，共 {len(intersection_portfolio)} 条记录。")

    # 4. 筛选回测周期
    start_date = pl.lit(config['BACKTEST_START_DATE']).str.to_date()
    end_date = pl.lit(config['BACKTEST_END_DATE']).str.to_date()
    intersection_portfolio = intersection_portfolio.filter(pl.col('date').is_between(start_date, end_date))

    # 5. 【核心向量化步骤】将交集组合与收益数据合并
    merged_df = intersection_portfolio.join(returns_df, on=['date', 'stkcd'], how='inner')

    # 6. 【核心向量化步骤】计算交集组合的等权月度收益
    portfolio_returns_df = merged_df.group_by('date').agg(
        pl.col('stock_return').mean().alias('portfolio_return')
    ).sort('date')

    # 填充缺失月份，确保时间序列连续
    if portfolio_returns_df.is_empty():
        print("警告：在指定的回测周期内没有生成任何投资组合收益。")
        return None, None

    full_date_range = pl.date_range(
        portfolio_returns_df.get_column('date').min(),
        portfolio_returns_df.get_column('date').max(),
        interval="1mo",
        eager=True
    ).alias("date")

    portfolio_returns_df = pl.DataFrame(full_date_range).join(
        portfolio_returns_df, on='date', how='left'
    ).fill_null(0.0)

    print(f"向量化回测完成，已生成 {len(portfolio_returns_df)} 条月度收益记录。\n")

    return portfolio_returns_df, benchmark_df


# =================================================================== #
#                   【2. 绩效计算与保存 (Polars)】                    #
# =================================================================== #

def calculate_performance_and_save_polars(portfolio_returns_df, benchmark_df, config):
    """为交集组合计算绩效并保存结果。"""
    print("--- 步骤 3: 计算并保存交集组合的绩效 ---")

    output_dir = config['OUTPUT_DIR']
    output_dir.mkdir(parents=True, exist_ok=True)
    returns_output_file = output_dir / f"{config['STRATEGY_NAME']}_returns.csv"
    performance_output_file = output_dir / f"{config['STRATEGY_NAME']}_performance.csv"

    risk_free_rate = config['RISK_FREE_RATE']

    # 合并基准收益
    final_returns_df = portfolio_returns_df.join(benchmark_df, on='date', how='left').fill_null(0)

    # 计算组合和基准的累计收益
    final_returns_df = final_returns_df.with_columns(
        (1 + pl.col('portfolio_return')).cum_prod().alias('cumulative_portfolio_return'),
        (1 + pl.col('benchmark_return')).cum_prod().alias('cumulative_benchmark_return')
    )

    # 计算各项绩效指标
    total_months = len(final_returns_df)
    if total_months == 0:
        print("错误：无法计算绩效，因为没有有效的月度收益数据。")
        return

    # 计算组合指标
    portfolio_perf = final_returns_df.select(
        pl.col('portfolio_return').alias('return'),
        pl.col('cumulative_portfolio_return').alias('cum_return'),
        (pl.col('portfolio_return') - pl.col('benchmark_return')).alias('excess_return')
    ).select(
        annualized_return = ((pl.col('cum_return').last()) ** (12 / total_months) - 1),
        annualized_volatility = (pl.col('return').std() * np.sqrt(12)),
        max_drawdown = (((pl.col('cum_return') / pl.col('cum_return').cum_max()) - 1).min()),
        tracking_error = (pl.col('excess_return').std() * np.sqrt(12))
    ).row(0, named=True)

    # 计算基准指标
    benchmark_perf = final_returns_df.select(
        pl.col('benchmark_return').alias('return'),
        pl.col('cumulative_benchmark_return').alias('cum_return')
    ).select(
        annualized_return = ((pl.col('cum_return').last()) ** (12 / total_months) - 1),
        annualized_volatility = (pl.col('return').std() * np.sqrt(12)),
        max_drawdown = (((pl.col('cum_return') / pl.col('cum_return').cum_max()) - 1).min())
    ).row(0, named=True)

    annualized_excess_return = portfolio_perf['annualized_return'] - benchmark_perf['annualized_return']
    tracking_error = portfolio_perf['tracking_error']
    information_ratio = annualized_excess_return / tracking_error if tracking_error != 0 else 0

    # 整理并合并结果
    all_metrics = [
        {
            'group': '交集策略 (0.2+0.1)',
            '年化收益率': portfolio_perf['annualized_return'],
            '年化波动率': portfolio_perf['annualized_volatility'],
            '夏普比率': (portfolio_perf['annualized_return'] - risk_free_rate) / portfolio_perf['annualized_volatility'] if portfolio_perf['annualized_volatility'] != 0 else 0,
            '最大回撤': portfolio_perf['max_drawdown'],
            '累计收益率': final_returns_df['cumulative_portfolio_return'].last() - 1,
            '年化超额收益率': annualized_excess_return,
            '信息比率': information_ratio,
            '跟踪误差': tracking_error
        },
        {
            'group': '基准 (沪深300)',
            '年化收益率': benchmark_perf['annualized_return'],
            '年化波动率': benchmark_perf['annualized_volatility'],
            '夏普比率': (benchmark_perf['annualized_return'] - risk_free_rate) / benchmark_perf['annualized_volatility'] if benchmark_perf['annualized_volatility'] != 0 else 0,
            '最大回撤': benchmark_perf['max_drawdown'],
            '累计收益率': final_returns_df['cumulative_benchmark_return'].last() - 1,
            '年化超额收益率': None, '信息比率': None, '跟踪误差': None
        }
    ]
    performance_df = pl.DataFrame(all_metrics)

    final_returns_df.write_csv(returns_output_file, float_precision=6)
    print(f"\n交集组合的月度收益率详情已保存至: {returns_output_file}")
    performance_df.write_csv(performance_output_file, float_precision=6)
    print(f"交集组合的绩效指标已保存至: {performance_output_file}")
    print(f"\n--- {config['STRATEGY_NAME']} 绩效简报 ---")
    print(performance_df)


# =================================================================== #
#                          【3. 主函数执行】                          #
# =================================================================== #

def main(config):
    """主执行函数"""
    start_time = time.time()
    portfolio_returns_df, benchmark_df = run_intersection_backtest_polars(config)
    if portfolio_returns_df is not None:
        calculate_performance_and_save_polars(portfolio_returns_df, benchmark_df, config)
    end_time = time.time()
    print(f"\n--- 所有任务完成！总耗时: {end_time - start_time:.2f} 秒 ---")

# =================================================================== #
#                       【4. 脚本执行入口】                              #
# =================================================================== #

if __name__ == "__main__":

    # --- 定义 pbroe7.4 (0.2+0.1)交集策略回测的配置 ---
    CONFIG_INTERSECTION_20_10 = {
        # --- 策略与输出配置 ---
        "STRATEGY_NAME": "pbroe7.4_intersection_20_10_backtest",
        "OUTPUT_DIR": Path("E:/PBROE/ch7pl/backtest_results_polars"),

        # --- 输入文件路径 ---
        "RESIDUAL_FILE": Path("E:/PBROE/ch7pl/pbroe7.1_residuals_and_quantiles_pure_polars.csv"),
        "RETURNS_FILE": Path("E:/PBROE/data/TRDNEW_Mnth.csv"),
        "BENCHMARK_FILE": Path("E:/PBROE/data/benchmark_indices.csv"),

        # --- 通用回测参数 ---
        "BACKTEST_START_DATE": '2010-05-01',
        "BACKTEST_END_DATE": '2025-04-30',
        "BENCHMARK_CODE": '000300',
        "RISK_FREE_RATE": 0.03
    }

    # --- 执行回测 ---
    main(CONFIG_INTERSECTION_20_10)


--- 步骤 1: 加载数据 (Polars) ---
所有数据加载成功。

--- 步骤 2: 构建交集投资组合 ---
已构建 Z-Score 最低 20% 组合，共 105479 条记录。
已构建 Quantile 最低 10% 组合，共 83307 条记录。
已构建交集组合，共 26958 条记录。
向量化回测完成，已生成 180 条月度收益记录。

--- 步骤 3: 计算并保存交集组合的绩效 ---

交集组合的月度收益率详情已保存至: E:\PBROE\ch7pl\backtest_results_polars\pbroe7.4_intersection_20_10_backtest_returns.csv
交集组合的绩效指标已保存至: E:\PBROE\ch7pl\backtest_results_polars\pbroe7.4_intersection_20_10_backtest_performance.csv

--- pbroe7.4_intersection_20_10_backtest 绩效简报 ---
shape: (2, 9)
┌────────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬──────────┐
│ group      ┆ 年化收益  ┆ 年化波动  ┆ 夏普比率  ┆ … ┆ 累计收益  ┆ 年化超额  ┆ 信息比率 ┆ 跟踪误差 │
│ ---        ┆ 率        ┆ 率        ┆ ---       ┆   ┆ 率        ┆ 收益率    ┆ ---      ┆ ---      │
│ str        ┆ ---       ┆ ---       ┆ f64       ┆   ┆ ---       ┆ ---       ┆ f64      ┆ f64      │
│            ┆ f64       ┆ f64       ┆           ┆   ┆ f64       ┆ f64       ┆          ┆          │
╞════════════╪═══════════╪═══════════╪════